In [49]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

ERROR: Operation cancelled by user


In [50]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [57]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://juliedata.s3-us-west-2.amazonaws.com/WordsMasterDone.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("WordsMasterDone.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+
|          Statements|
+--------------------+
|Theres one thing ...|
|When you work wit...|
|We have a lot to ...|
|Through our insto...|
|SkyTouch Technolo...|
|For more informat...|
|Glynlyon Inc is o...|
|Picmonic brings t...|
|The Picmonic stor...|
|Established in 20...|
|Blue Yonder forme...|
|The future of log...|
|GoDaddy is the wo...|
|With a worldwide ...|
|Infusionsoft anno...|
|Well continue to ...|
|SiteLock is the G...|
|Nextiva is a lead...|
|AudioEye is a tec...|
|assisTek has been...|
+--------------------+
only showing top 20 rows



In [59]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['Statements']))
data_df.show()

+--------------------+------+
|          Statements|length|
+--------------------+------+
|Theres one thing ...|   293|
|When you work wit...|   265|
|We have a lot to ...|   489|
|Through our insto...|   287|
|SkyTouch Technolo...|   364|
|For more informat...|   178|
|Glynlyon Inc is o...|   909|
|Picmonic brings t...|   535|
|The Picmonic stor...|   570|
|Established in 20...|   141|
|Blue Yonder forme...|   551|
|The future of log...|   434|
|GoDaddy is the wo...|   395|
|With a worldwide ...|   577|
|Infusionsoft anno...|   648|
|Well continue to ...|   412|
|SiteLock is the G...|   522|
|Nextiva is a lead...|   610|
|AudioEye is a tec...|   462|
|assisTek has been...|   118|
+--------------------+------+
only showing top 20 rows



### Feature Transformations


In [65]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
  
# function to print sentiments 
# of the sentence. 
def sentiment_scores(sentence): 
  
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
  
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
      
    print("Overall sentiment dictionary is : ", sentiment_dict) 
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative") 
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral") 
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
  
    print("Sentence Overall Rated As", end = " ") 
  
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        print("Positive") 
  
    elif sentiment_dict['compound'] <= - 0.05 : 
        print("Negative") 
  
    else : 
        print("Neutral") 
  
  


     |████████████████████████████████| 133kB 2.8MB/s 


In [74]:
# Driver code 
print("\n1 statement :") 
sentence = "Theres one thing that unites everyone who works at PetSmart We all love pets Cats dogs goldfish parakeets hamsters you name it we love them Every day we bring our passion together to create something amazing a place for our many best friends to be healthy happy and to keep those tails wagging When you work with us youll discover that we love our people too Whether youre based in a store distribution center or at our home office youll find yourself inspired and empowered to go further reach higher and really make the most out of your career with PetSmart We have a lot to bark about We employ 55000 associates and operate 1600 retail stores in the United States Canada and Puerto Rico In additional to providing over 50000 products in our stores we also offer services such as our Grooming Salon PetsHotel PetSmart Pet Training and Everyday Adoption Centers Our portfolio of digital resources for pet parentsincluding PetSmartcom Chewycom and OnlyNaturalPetcomoffers the most comprehensive online pet supplies and pet care information in the US Through our instore pet adoption partnership with independent nonprofit organizations PetSmart Charities® and PetSmart Charities™ of Canada PetSmart has helped 8 million pets find their forever homes and have donated over $310 million in grants to over 3500 animal welfare organizations" 
# function calling 
sentiment_scores(sentence) 
  
print("\n2nd Statement :") 
sentence = "SkyTouch Technology is the provider of the most widely used cloudbased SaaS hotel operations platform Since launching in 2013 we as a team have reached major milestones and have had continuous growth year over year and we are now in more than 7000 hotels globally Together we work hard we laugh a lot we share ideas and we help empower each other and our customers For more information about career opportunities and to hear what some of our team members have to say about working at SkyTouch go to"
sentiment_scores(sentence) 
  
print("\n3rd Statement :") 
sentence = "Glynlyon Inc is on the cutting edge of innovation for education technology  empowering students school districts and families use of technology to learn teach and share at home school and in life First to market with solutions that address the challenges faced by its customers Glynlyon offers selfpaced student centered curriculum that enhances the capacity for districts to blend learning and empowers teachers to create lessons from a standards aligned curriculum Glynlyon offers industryleading support to its customers through custom implementation and training plans and direct instructional support for parents and students Glynlyons family of brands includes Odysseyware and Alpha Omega Publications and serves the consumer and institutional marketplace Its focus is designing education technology solutions that engage learners optimize teaching and learning and empower students to achieve their goals"
sentiment_scores(sentence) 

print("\n4 :")
sentence = "Picmonic brings the power of social visual platforms to learning We help people create share and explore highly effective audiovisual study cards called Picmonics “pictorial mnemonics” Picmonics are not only proven to increase memorization capacity and improve longterm recall of information they incorporate humor and unforgettable graphics to make learning fun The Picmonic vision is of a social educational platform where anyone can author share and discover Picmonics to improve academic professional and personal learning outcomes The Picmonic story began in medical education where tens of thousands of learners around the world are utilizing Picmonics to learn high stakes premedical medical and nursing content and increase their test scores  but that is just the beginning Millions of postsecondary and professional education students across the globe will learn with Picmonic and have fun while mastering information like never before Picmonic gives every student the power to create an engaging and personalized learning experience while building a connected and collaborative learning community Established in 2011 Picmonic Inc is headquartered in Tempe AZ Founded by two medical students and backed by Matt Pittinsky and Bobby Babbrah"
sentiment_scores(sentence)

print("\n5 :")
sentence = "Blue Yonder formerly JDA Software provides seamless frictionfree commerce empowering every organization and person on the planet to fulfill their potential Blue Yonders machine learningdriven digital fulfillment platform enables clients to deliver to their customers when how and where they want it Applying over 35 years of domain expertise contextual intelligence and data science Blue Yonder is helping more than 3000 of the worlds leading manufacturers retailers and logistics companies create more autonomous sustainable and profitable operations"
sentiment_scores(sentence)

print("\n6 :")
sentence = "The future of logistics is technology and GlobalTranz is leading the way forward We are a community of people who work every day to improve the way freight is moved from one place to the next We live and breathe our values and bring out the best in each other through positivity collaboration and a passion for problemsolving Thats what makes working here so specialwith over 1000 employees we make things happen and have fun doing it"
sentiment_scores(sentence)

print("\n7 :")
sentence = "GoDaddy is the worlds largest web services platform and most outspoken champion for everyday entrepreneurs around the globe Were on a mission to empower entrepreneurs everywhere by giving them all the help and tools they need to grow online In doing so were making opportunity more inclusive for all and fueling a new generation of entrepreneurial endeavors  be they commercial civic or creative With a worldwide community of customers 19 million strong GoDaddy is the place people come to name their idea create a compelling brand and an awesome website attract customers with digital and social marketing and manage their work Where no tool alone will suffice we give our customers direct oneonone guidance with a human connection that is unlike any other technology company in the industry Job One for GoDaddy is to empower our community of independentlyminded individuals both practically and emotionally on their journey rendering them more confident proud and strong"
sentiment_scores(sentence)

print("\n8 :")
sentence = "Infusionsoft announced today it has rebranded the company as Keap a name that represents the very essence of the grit and passion small businesses put in each day to keep going keep serving and keep growing With the rebrand comes the launch of a new product also named Keap the industrys first smart client management software built specifically for small businesses in the home personal and professional service industries For 17 years Infusionsoft has led the market as the pioneer of CRM and marketing automation software for small businesses The Keap product launch and company rebrand is a signal the company is expanding to serve a new market Well continue to lead the CRM and marketing automation industry with exciting updates coming this year with what we are now calling Infusionsoft by Keap” says Keap COO Keith Reed “With the introduction of our new Keap product we are able to serve an even larger market of small service providers who have been shut out of the benefits of automation because software providers have made it too hard and expensive"
sentiment_scores(sentence)

print("\n9 :")
sentence = "SiteLock is the Global Leader in business website security and is the only web security solution to offer complete cloudbased website protection We find and fix threats prevent future attacks accelerate website performance and meet PCI compliance standards for businesses of all sizes basically like Batman if Gotham City was the internet Founded in 2008 SiteLock now protects over 15 million websites worldwide Our mission is to protect every website on the internet yes we know thats a colossal goal but we are THAT good"
sentiment_scores(sentence)

print("\n10 :")
sentence = "Nextiva is a leading provider of cloudbased business communication solutions Its dynamic easytouse tools are powered by NextOS the allinone communication platform The result? A unified communications solution that delivers realtime insight into the customer journeyNextiva has won multiple product and company culture awards along the way As a company Nextiva is dedicated to providing Amazing Service  both to its customers and its employees Learn about careers at Nextiva http//nextivacom/careersNextiva has developed a solution for the future One platform one workspace zero friction Tomas Gorny Nextiva CEO"
sentiment_scores(sentence)

print("\n11 :")
sentence = "AudioEye is a technology company serving businesses committed to providing equal access to their online content Transforming how the world experiences digital content AudioEye solves issues of accessibility and enhances the user experience for the broadest audience possibleSpecializing in speed to compliance with ADArelated digital accessibility requirements AudioEye offers technology that makes digital content more accessible and more usable for more people"
sentiment_scores(sentence)

print("\n12 :")
sentence = "assisTek has been providing successful eSource solutions for clinical trials for over 22 years These solutions include SiteBased eCOA  electronic collection of patient  and clinician assessments at the study site FieldBased ePRO  customized data capture of patientreported data for patients at home or anywhere Direct Clinical Data Capture capture of clinicianreported data for a clinical trial and transmission of this  data to an EDC system assisTeks systems can be delivered on any Android device including Tablets & Smartphones  Data transmission is accomplished wirelessly either via cellular network or WIFI so that patients & study teams do not have to connect any cradles modems or wires   "
sentiment_scores(sentence)

print("\n13 :")
sentence = "WE SIMPLIFY THE DATA CENTER AND MAKE IT SMART For the first time enterprises governments and service providers have a technology platform that is right for their needs Our goal is to revolutionize the performance of the data center through advanced modular technology infrastructure management software and big data analytics Right size Right place Right impact Softwaredefined modular data center infrastructure that can be dynamically provisioned based on the needs of your application Standardized yet customizable BASELAYER™Anywhere is IT agnostic and provides you with the flexibility and energy efficiency you need to deliver performance and reduce cost Connect collect and analyze all of your missioncritical assets across multiple locations for comprehensive visibility and control BASELAYER RunSmart provides you with a unified operating system so you can visualize and evaluate infrastructure performance  from any device anywhere any time"
sentiment_scores(sentence)

print("\n14a :")
sentence = "CampusLogic started in 2011 with the mission to help schools change lives We are passionate about transforming the student financial aid journey into the student financial success journey and we do that by offering innovative solutions that empower institutions to remove barriers for students When financial aid is easy mobile and personalized more students can access itand the education it pays for Students can make more informed borrowing decisions and better navigate financial aid processes Our platform frees up time for financial aid staff to focus less on routine tasks and more on what matters most advising students You can learn more about the company and its awesome culture at campuslogiccom "
sentiment_scores(sentence)

print("\n15b :")
sentence = "CompuGroup Medical is a global eHealth provider with a comprehensive portfolio of cuttingedge IT solutions for the healthcare industry We connect doctors hospitals community health facilities dentists pharmacists health insurers and other service providers to create an integrated network of all healthcare stakeholders As a result we can help these organizations improve efficiency optimize care and increase patient satisfaction while also improving profitability Our marketleading solutions include everything from electronic health records to innovative practice management systems These solutions are present in over 400000 practices and healthcare organizations worldwide We are also proud to maintain and sustain longterm relationships with our clients many of whom have been using our products for decades Our dedicated knowledgeable team consistently delivers innovative products and services with the customer in mind knowing that in addition to running an effective business their main focus is to heal the patient We share that same passion and it drives everything we do As an ownerled and publicly traded company CGM uniquely combines the personal touch of a local business with the strength of a global entity We are committed to meeting and exceeding our clients expectations and our goal is to become the #1 choice for all healthcare providers in the United States  a position we have already achieved in many other countries around the world"
sentiment_scores(sentence)

print("\n16 :")
sentence = "Dash is a digital currency that enables anyone anywhere in the world to make quick easy and cheap payments at any time without going through a central authority Based on a decentralized peertopeer network and secured by strong cryptography Dash offers a safe and userfriendly payment method without barriers Dash is portable inexpensive divisible and fast digital cash for both the internet and everyday lifeFor press and media inquiries please contact the Dash press team at newsroom@dashorg We look forward to hearing from you"
sentiment_scores(sentence)

print("\n17 :")
sentence = "Design Pickle is the #1 flatrate graphic design service in the world  It is a cost effective reliable and easy to use graphic design solution for any entrepreneur business and creative team We are a flat rate graphic design and custom illustration subscription service"
sentiment_scores(sentence)


print("\n18 :")
sentence = "Digital Air Strike provides the most complete social media marketing reputation management intelligent lead response and consumer engagement solution Digital Air Strike spots trends shares best practices and takes action on behalf of clients The key differentiator is that Digital Air Strike has a dedicated team and awardwinning technology helping 5000 businesses worldwide engage with their customers online Digital Air Strike offers numerous benefits and generous time off to its 200 employees and is consistently rank a Top Workplace Our ethos is about doing whatever it takes to help colleagues and clients achieve their goals and creating an internal community that feels less corporate and more familial Whether its the “promote from within” culture or the “Whatever It Takes” awardswhere employees nominate each otherDigital Air Strike is committed to fostering shared success The company and our employees support causes ranging from wildlife sanctuaries to single parents battling cancer We give back financially and through volunteerism to dozens of charities such as American Cancer Society East Valley Womens League and the YMCA"
sentiment_scores(sentence)

print("\n19 :")
sentence = "Edgenuity partners with schools and districts to provide online and blended learning solutions that propel student success and empower every teacher to deliver more effective instruction Edgenuity Courseware offers rigorous courses for initial credit and credit recovery in ELA math science social studies as well as world languages electives CTE dual credit and AP® For our youngest learners Hybridge offers online content that supplements classroom instruction to help students master gradelevel skills in reading and math To help students close gaps Pathblazer for elementary school and MyPath for middle and high school provide datadriven personalized intervention so students can catch up keep up or get ahead UpSmart for middle school is a new test readiness program that ensures all students can demonstrate mastery of state standards in ELA and math while Edgenuity Virtual Tutors for high school offer comprehensive preparation for endofcourse and exit exams as well as the SAT® ACT® PSAT™ GED® HiSET® and TASC™ To further support our partnership with schools Edgenuity Instructional Services offers highly qualified statecertified virtual instructors who can serve as the teacher of record And our Teacher and Learning Services provide professional development workshops that build the capacity of teachers and administrators"
sentiment_scores(sentence)

print("\n20 :")
sentence = "Reduce shipping costs with our Digital Freight Marketplace giving you access to the lowest rates on live capacity with just a few clicks Expand your partner network by connecting to thousands of verified carriers and brokers to make sure your loads are always covered Our onetime setup gets you connected fast saving you the time and hassle of onboarding multiple partners  weve got you covered on everything from billing to cargo insurance "
sentiment_scores(sentence)

print("\n21 :")
sentence = "Enghouse Interactive delivers technology and expertise to maximize the value of every customer interaction with products spanning multichannel call centers CTI integration IVR / SelfService operator attendant consoles call recording and quality monitoring outbound predictive dialer knowledge management solutions and more Our comprehensive portfolio of interaction management solutions are resilient and scalable with a flexible deployment model to grow with your business Enghouse Interactive is a proud partner to key unified communications and telephony suppliers including Microsoft NEC Cisco Avaya ShoreTel and Mitel With a worldwide ecosystem of contact center service providers system integrators channel partners and tens of thousands of agents handling millions of daily customer interactions worldwide Enghouse Interactive solutions are preferred by mid to large enterprises telecom carriers business process outsourcers and contact center service providers"
sentiment_scores(sentence)

print("\n22 :")
sentence = "We are kicking off a telematics revolution in the construction industry and beyond Our company is selffunded  Decisions can be made in minutes We are looking for highly talented people to help us expand our newest startup product  Fleet IntelligenceFI  FI a telematics solution is less than two years old and has already received major industry endorsements Rapid innovation in the product and processes means we are doing many things that have not been done before  To facilitate FI growth we need to constantly evolve our product processes systems messaging methodologies strategies and tactics  Priorities get rearranged every day We thrive on creativity innovation problem solving and energy drinks provided by company of course Were building a mighty machine to process massive data in a world where a 01 second lag is too great  The upside potential is enormous the timing is perfect the work is hard and the pace is fast  As a team we accomplish more in one day than what the average team can accomplish in one week while having a great time  We are on the journey together and each person is able to place their footprint on history in the making  If reaching Big Hairy Audacious Goals BHAGs gets your adrenaline pumping come join us"
sentiment_scores(sentence)

print("\n23 :")
sentence = "InkSoft the next generation of sales and production software for decoratorsSell smarter Streamline production Grow profitsInkSoft saves decorators time and increases their profits with a unified software system that ensures a great customer experience worryfree order processing and production/delivery of quality products This smart platform manages everything from marketing to sales to order processing to production Shop efficiency customer satisfaction and profits go up Headaches mistakes and wasted time go away Weve heard over and over again that decorators dont have business software built for their specific needs and challenges We listened and we created InkSoft to be that platform And it is revolutionary Never before has a decorator had all the tools in one place to sell manage and deliver orders We leverage smart technology to help manage the entire decoration process from customer acquisition to production to repeat business InkSoft was made to give decorators more profits more time and more confidence This is the first system built entirely to solve the needs of decorators wanting to modernize their business and leapfrog the competition"
sentiment_scores(sentence)



1 statement :
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'compound': 0.9949}
sentence was rated as  0.0 % Negative
sentence was rated as  78.4 % Neutral
sentence was rated as  21.6 % Positive
Sentence Overall Rated As Positive

2nd Statement :
Overall sentiment dictionary is :  {'neg': 0.014, 'neu': 0.832, 'pos': 0.153, 'compound': 0.9136}
sentence was rated as  1.4000000000000001 % Negative
sentence was rated as  83.2 % Neutral
sentence was rated as  15.299999999999999 % Positive
Sentence Overall Rated As Positive

3rd Statement :
Overall sentiment dictionary is :  {'neg': 0.01, 'neu': 0.832, 'pos': 0.158, 'compound': 0.9524}
sentence was rated as  1.0 % Negative
sentence was rated as  83.2 % Neutral
sentence was rated as  15.8 % Positive
Sentence Overall Rated As Positive

4 :
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.823, 'pos': 0.177, 'compound': 0.9794}
sentence was rated as  0.0 % Negative
sentence was rated as  82.3 % Neutral
sent

In [77]:
print("\n24 :")
sentence = "Since 1983 Intellimed has been at the forefront of healthcare analytics providing services and solutions that give healthcare providers the tools they need to make datadriven decisions Led by a team with decades of experience in healthcare management data analysis and market research Intellimed offers clients actionable intelligence and unparalleled support Intellimeds premier products IntelliClient™ and IntelliMarket™ allow organizations to supplement their decisionmaking with strong extensive data that provides a clear picture of their market Headquartered in Phoenix Ariz Intellimed serves leading healthcare companies throughout the US including hospitals and health systems healthcare providers payors and more "
sentiment_scores(sentence)

print("\n25 :")
sentence = "Offerpad is a leading techenabled real estate company The companys ondemand platform is revolutionizing the industry with an innovative endtoend real estate experience for those looking to buy or sell a home Offerpad streamlines the process for consumers making the process convenient and hasslefreeFounded in 2015 the companys leadership team has decades of real estate experience and collectively has purchased renovated and rented or sold more than 100000 homesOfferpad is a privatelyheld company headquartered in Chandler Arizona and currently operates in several markets across the country collectively reaching more than 441 cities with additional cities being added in the coming months"
sentiment_scores(sentence)

print("\n26 :")
sentence = "Our mission is to ensure integrity in all learning environments and developing new technologies to support innovation in education and beyond Using stateoftheart technology and endtoend data security Proctorio ensures the total learning integrity of every assessment every time Our software eliminates human error bias and much of the expense associated with remote proctoring and identity verification Proctorio also integrates effortlessly with whatever learning management system you already useno scheduling waiting or additional logins requiredfor instant objective secure results Still not sure? Give Proctorio a test"
sentiment_scores(sentence)

print("\n27 :")
sentence = "RevolutionParts launched in 2013 with a simple goal use our decades of experience to revolutionize an archaic industry providing a better way for our customers to help people stay mobile through selling parts online Make the process dramatically more efficient Make it  Read more We are RevolutionParts and we are on a mission to disrupt and dominate the auto parts eCommerce industry by providing our customers a simple yet cuttingedge solution so they can thrive in a highly competitive online market Our focus is to build THE industryleading platform that keeps customers coming back We believe that things are better when each and every one of us is working towards the same set of goals and we are beyond excited that youd consider joining us on our mission"
sentiment_scores(sentence)

print("\n28 :")
sentence = "CellTrust Corporation founded in 2006 and headquartered in Scottsdale Arizona USA is a global leader in compliant mobile communications archiving and eDiscovery for the highly regulated financial government and healthcare industriesCellTrusts flagship SL2 compliant text messaging and voice platform helps those operating in highly regulated industries to enforce mobile communication compilance while balancing mobile productivity risk and control DoddFrank Act GLBA SOX FINRA SEC FCA MiFID II FOIA Sunshine Laws HIPAA CellTrusts Patented SecureSMS and SecureVoice products support secure enterprise mobile communication for those companies wanting to communicate within a secure and protected environment"
sentiment_scores(sentence)

print("\n29 :")
sentence = "ClearVoice is an unprecedented leadingedge platform that provides a fullcircle solution to content marketing We believe a clear voice is the most powerful voice no matter its volume or medium So we live to help you express yours Its the ClearVoice wayBy connecting you with worldclass freelance talent through our easytouse collaborative platform we can help your brand create content that empowers delights and activates With ease With purpose With successExplore the platform at clearvoicecom"
sentiment_scores(sentence)

print("\n30 :")
sentence = "Anju Software is a leading provider of comprehensive software solutions to the life sciences industry Anju provides an integrated platform spanning clinical operations and medical affairs Anju has combined deep industry and software domain expertise to build a next generation platform for the life sciences sector that can scale to provide solutions that meet customers needs from molecule to retirement Anju provides endtoend software solutions that help customers manage missioncritical pharma processes and turn data into actionable insights Anjus philosophy is to support and guide clients over the long term in meeting their evolving software needs Anju is a portfolio company of Abry Partners which is one of the most experienced and successful sectorfocused private equity investment firms in North America Since their founding in 1989 the firm has completed over $80 billion of leveraged transactions and other private equity or preferred equity placements Currently the firm manages over $50 billion of capital across their active funds Anjus mission is to complement its strong product offering with exceptional customer support and delivering true quantifiable value"
sentiment_scores(sentence)

print("\n31 :")
sentence = "Welcome to Axosoft Were an organization made up of around 50 or so amazing people mostly devs We love to work on challenging projects write beautiful code and build software solutions that help transform the world Were the creators of Axosoft our beautiful agile project management software; GitKrakren our elegant crossplatform Git client; as well as #ItWasNeverADress a campaign that is shifting perceptions about women in the tech industry and beyond We develop software that is used by the worlds most elite software engineers Companies like Apple Google Microsoft Amazon and thousands of other leading companies use GitKraken Glo Boards and Axosoft We are looking for top talent to help us make Axosoft an even better company"
sentiment_scores(sentence)

print("\n32 :")
sentence = "Small businesses can generate more leads and drive sales with Pure Chat When companies add Pure Chat to their website they can message multiple leads and customers at onceinstead of spending hours talking on the phone They can also chat with new website visitors at their desktop or on the go with our mobile apps Pure Chat is simple and intuitive unlike other enterprisefocused chat software companies Get started today for freeSince launching in 2012 small businesses have generated over 28 Million chats on the Pure Chat platform In April 2015 Pure Chat secured $15M in seed funding to fuel growth Pure Chat made with love by a group of highly caffeinated nerf gun aficionados in sunny Scottsdale AZ"
sentiment_scores(sentence)

print("\n33 :")
sentence = "RallyUp provides charities schools and other organizations with an online platform to run nextgeneration fundraising experiences that help them further their causes in significant ways"
sentiment_scores(sentence)

print("\n34 :")
sentence = "At ShiftX we love to make peoples lives better every day With ShiftX you give your employees a voice to create  better work and vacation schedules Your seniority Shift preferences and buddy preferences are all used to get you the best possible schedule ShiftX makes scheduling easy fair transparent and delightful at scale Whether you live in a complex regulatory environment or just want to create schedules that fit your employees lives ShiftX makes it easy We designed the ShiftX platform to solve all of your scheduling problems This extensible platform enables you to customize your unique environment provides APIs/plugins for integration and can take advantage of our scheduling AI plugins We make it easy for you to optimize your scheduling processes"
sentiment_scores(sentence)

print("\n35 :")
sentence = "In the early days of broadband Internet downloading of multigigabyte online games was an excruciating experience In 2005 our founding team knew a thing or two about download technologies content delivery networks CDNs and game development We also knew that this was a problem that shouldnt have to be solved many times over by every game studio especially if it meant diverting precious development resources that from working on creating the next great game We searched for every way possible to optimize the player onboarding process for the benefit of game developers online publishers and players Our goal was to eliminate as much of the friction that stands between the player and their gameplay as possible Along the way we have had the great fortune of working with some of the industrys top game studios as well as major software publishers that needed a solution to this problem These valuable relationships helped to drive our technology development and feature roadmap As we helped our customers deploy their software applications to hundreds of millions of systems worldwide our technology hardened the addressable use cases expanded We stopped keeping track of our aggregate when we reached over 100 billion software downloads and updates In 2016 we added the scalability of AWS global cloud infrastructure to bring ondemand compute and automated workflows to our customers to speed their deployment pipeline and eliminate CapEx costs In 2017 as enterprise companies began embracing game technologies for virtual augmented and mixed reality XR in business we added enterprise features and security to the DIRECT platform Today our mission is to enable continuous delivery practices beyond the edge into corporate educational and lastmile networks Our commitment to innovation ensures our customers that they have solutions needed today and the peace of mind to know we will grow with them for their future"
sentiment_scores(sentence)

print("\n36 :")
sentence = "At Virtuous were committed to helping nonprofits reimagine generosity We believe that giving is about personal connections not sales transactions Our generosity is driven by our passions and relationships  and givers want to feel like they are part of a movement bigger than themselvesThats why were building fundraising software that nonprofits love We put the joy back in fundraising so our customers can provide even more good in the world"
sentiment_scores(sentence)

print("\n37 :")
sentence = "We sell cars but were not car salespeople is a popular refrain at Carvana Instead we provide a seamless online car buying experience for customers from coasttocoast Carvana allows customers to browse finance and purchase more than 15000 vehicles from the comfort of home or on the go via their mobile device while also offering the convenience of selling your car to us without ever having to step foot inside a dealership Customers who purchase with Carvana enjoy the peace of mind that comes with our 7day return policy and the sense of security in knowing that each and every Carvana vehicle is Carvana Certified having passed a rigorous 150point inspection and has never been in a reported accident And with the option to choose assoonas nextday delivery or pick up from one of our signature Car Vending Machines the days of spending a Saturday at the dealership are officially over"
sentiment_scores(sentence)

print("\n38 :")
sentence = "ON Semiconductor Nasdaq ON is a Fortune 500 company driving energy efficient innovations empowering customers to reduce global energy use The company is a leading supplier of semiconductorbased solutions offering a comprehensive portfolio of energy efficient power and signal management logic standard and custom devices The companys products help engineers solve their unique design challenges in automotive communications computing consumer industrial medical and military/aerospace applications ON Semiconductor operates a responsive reliable worldclass supply chain and quality program and a network of manufacturing facilities sales offices and design centers in key markets throughout North America Europe and the Asia Pacific regions"
sentiment_scores(sentence)

print("\n39 :")
sentence = "Benchmark and its family of brands to include Secure Technology Lark RF Technology and Precision Technologies is a worldwide provider of innovative product design engineering services technology solutions and advanced manufacturing services From initial product concept to volume production including direct order fulfillment and aftermarket services Benchmark has been providing integrated services and solutions to original equipment manufacturers since 1979 Today Benchmark proudly serves the following industries aerospace defense medical technologies complex industrials test and instrumentation nextgeneration telecommunications and highend computing Benchmarks global operations network includes facilities in eight countries and common shares trade on the New York Stock Exchange under the symbol BHEThe Benchmark team consists of ~13000 employees in 23 worldwide locations and is headquartered in Tempe Arizona"
sentiment_scores(sentence)

print("\n40 :")
sentence = "Sprouts one of the fastestgrowing retailers in the country has made healthy living accessible to shoppers for nearly two decades by offering affordable fresh natural and organic products True to its farmers market heritage Sprouts is known for pioneering its unique grocery model by offering a welcoming store layout featuring fresh produce at the center of the store an expansive bulk foods section and a vitamin department focused on overall wellness Sprouts also offers a unique assortment of healthier products with special attributes such as plantbased glutenfree ketofriendly and grassfed to meet the growing and diverse needs of todays consumer One thing is for sure working at Sprouts is a lot of fun Sounds right up your alley? All you need is a passion for people and flair for food We can show you the rest"
sentiment_scores(sentence)

print("\n41 :")
sentence = "At Shamrock Foods Company people come first  our associates our customers and the families we serve across the nation A privatelyheld familyowned and operated Forbes 500 company Shamrock is an innovator in the food industry and has been since being founded in Arizona in 1922 Shamrock specializes in the manufacturing and distribution of quality food and foodrelated products through a family of companies including Shamrock Foods foodservice distribution and Shamrock Farms dairy manufacturingThrough its family of companies Shamrock employs more than 4000 associates nationwide The companys success is built on three unwavering principles set forth by our founder WT McClelland in 1922  an unrelenting quality in people and products and service an innovative approach to business and a commitment to the community Learn more about our products philosophy and current job openings at wwwshamrockfoodscom  Mission At Shamrock Foods Company we live by our founding familys motto to treat associates as family and customers and suppliers as friends"
sentiment_scores(sentence)

print("\n42 :")
sentence = "Since 1946 Fender has revolutionized music and culture as one of the worlds leading musical instrument manufacturers marketers and distributors Fender Musical Instruments Corporation FMIC whose portfolio of brands includes Fender® Squier® Gretsch® guitars Jackson® EVH® and Charvel® follows a playercentric approach to crafting the highest quality instruments and musical solutions across genres FMIC is dedicated to unlocking the power of music through electric and acoustic guitars amplifiers pro audio accessories and digital products that inspire and enable musical expression at every stage from beginners to historymaking legends"
sentiment_scores(sentence)

print("\n43 :")
sentence = "EXOS is a leader in the field of human performance a category it created more than 15 years ago Today EXOS employs more than 3500 people in over 400 locations worldwide With awardwinning facilities technology and services EXOS helps people take control of their health and performance so they can achieve what matters most to them EXOS is trusted by hundreds of clients including onequarter of Fortune 100 companies world champions in sports and leaders in health care military and community organizations "
sentiment_scores(sentence)

print("\n44 :")
sentence = "Axway gives heritage IT infrastructure new life helping more than 11000 customers worldwide build on what they already have to invent their digital journeys add new business capabilities and drive growth ambitions With the futureproof AMPLIFY™ enterprise integration platform  an innovative combination of APIs B2B integration content services community and digital ecosystems  we guide continuous innovation and enhance the customer experience faster and more securely than ever before In business for more than 20 years Axway Euronext AXWPA employs over 1800 people in 18 countries Learn more at axwaycom"
sentiment_scores(sentence)

print("\n45 :")
sentence = "In March of 1963 a oneman pool supply operation working out of a backyard shed in North Hollywood California began That journey continues today  54 years later  as Leslies Poolmart Inc “The Worlds Largest Retailer of Swimming Pool Supplies” With over 900 retail stores in 35 states plus a Commercial Division Mail Order and Ecommerce Group Leslies offers many career opportunities Leslies offers the best selection of pool and spa chemicals pool cleaners pool equipment cleaning accessories and pool inflatables and floats We have the solution to all your pool and spa needs Every store offers free instore water testing and free instore labor on repairs  "
sentiment_scores(sentence)

print("\n46 :")
sentence = "Viad pronounced VEEahd makes sure conventiongoers get to their events Viad offers convention and event services exhibit design and construction and travel and recreation services Its event services operations are organized under its Global Experience Specialists GES brand which provides convention services to trade associations and exhibitors It also offers custom exhibit designers for corporations museums trade shows and retail stores Viads Travel & Recreation Group operates through its Brewster unit Canadian accommodations and tour provider and Glacier Park division mountain lodge operator The firm has operations across Canada Germany the UK the United Arab Emirates and the US"
sentiment_scores(sentence)

print("\n47 :")
sentence = "Bashas is working up a sweat standing its ground in Southwest The regional grocery chain operates about 130 stores down from more than 160 in 2008 all but two of which are located in Arizona The other locations are in California and New Mexico Its holdings include Bashas traditional supermarkets AJs Fine Foods gourmetstyle supermarkets and about a dozen Food City supermarkets which cater to Hispanics in southern Arizona It also runs a handful of Dine Markets in the Navajo Nation dine means the people in Navajo and Phoenix wine retailer Sportsmans Fine Wine and Spirits Founded in 1932 family owned Bashas emerged from Chapter 11 bankruptcy in August 2010 after closing stores"
sentiment_scores(sentence)

print("\n48 :")
sentence = "Headquartered in Tempe Arizona DriveTime is the largest privately owned used car sales and finance company in the country With more than 130 dealerships 3 operations centers and 15 inspection centers across the nation our 5000 employees are focused on getting the right customer in the right car at the right terms Starting with selection at auction DriveTime sources over 200 different vehicle makes and models Each vehicle is then sent to the inspection centers for reconditioning before being delivered to the dealership Once on the lot our nohaggle pricing model and belief in transparency provide a pressurefree sales environment for every customer Using our proprietary scoring technology and financing model we help our customers get into the car thats right for them regardless of their credit situation With over 25 years of industry experience and our dedication to streamlining the purchase process were redefining what it means to buy and finance a used car"
sentiment_scores(sentence)

print("\n49 :")
sentence = "Were the leading name in massage and collectively across franchised locations Massage Envy is Americas number 1 skin care provider While that may describe us it doesnt define us At its core the Massage Envy franchise network is a community of artists healers and wellness professionals united in one purpose helping people feel their best through regular massage and skin care so they can do more of what they love And with entire teams to help keep schedules full and treatment rooms ready massage therapists estheticians and stretch providers in Massage Envy franchised locations get to do more of what they love too MASSAGE ENVY FRANCHISED LOCATIONS* OFFER MORE WAYS TO FEEL GOOD WHILE DOING GOOD with selfcare resources for both body and mind exciting new services and enhancements and the opportunity to positively impact the lives of more than 15 million clients each month MORE FUN THAN A TEAM OF ONE with the support of not just a franchised location team but an entire community of more than 35000 therapists estheticians stretch providers and associates who share your passion A STRONG COMMITMENT TO SAFETY and setting the industry standard on providing a safe therapeutic healing environment for service providers clients and everyone in between *Each individual franchised location not ME SPE Franchising LLC MEF or any of its affiliates is the sole employer for all positions posted by a franchised location and each individual franchised location is not acting as an agent for MEF or any of its affiliates Hiring criteria benefits and compensation are set by each individually owned and operated franchised location and may vary from location to location "
sentiment_scores(sentence)

print("\n50 :")
sentence = "FreeportMcMoRan Inc FCX is a leading international mining company with headquarters in Phoenix Arizona FCX operates large longlived geographically diverse assets with significant proven and probable reserves of copper gold and molybdenum FCX is the worlds largest publicly traded copper producer FCXs portfolio of assets includes the Grasberg minerals district in Indonesia one of the worlds largest copper and gold deposits and significant mining operations in North and South America including the largescale Morenci minerals district in Arizona and the Cerro Verde operation in Peru FCX has a strong commitment to safety performance environmental management and the local communities where it operates FCX is a founding member of the International Council on Mining and Metals ICMM and is committed to implementation of the ICMM Sustainable Development Framework Our stock trades on the New York Stock Exchange under the ticker symbol “FCX”"
sentiment_scores(sentence)



24 :
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.861, 'pos': 0.139, 'compound': 0.9231}
sentence was rated as  0.0 % Negative
sentence was rated as  86.1 % Neutral
sentence was rated as  13.900000000000002 % Positive
Sentence Overall Rated As Positive

25 :
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.955, 'pos': 0.045, 'compound': 0.5719}
sentence was rated as  0.0 % Negative
sentence was rated as  95.5 % Neutral
sentence was rated as  4.5 % Positive
Sentence Overall Rated As Positive

26 :
Overall sentiment dictionary is :  {'neg': 0.06, 'neu': 0.758, 'pos': 0.181, 'compound': 0.9025}
sentence was rated as  6.0 % Negative
sentence was rated as  75.8 % Neutral
sentence was rated as  18.099999999999998 % Positive
Sentence Overall Rated As Positive

27 :
Overall sentiment dictionary is :  {'neg': 0.011, 'neu': 0.848, 'pos': 0.141, 'compound': 0.9459}
sentence was rated as  1.0999999999999999 % Negative
sentence was rated as  84.8 % Neutral
sentence was rated as

In [79]:
print("\n51 :")
sentence = "TPI is the largest USbased independent manufacturer of composite wind blades for the highgrowth wind energy market supporting global wind turbine manufacturers TPI operates composite products manufacturing facilities in North America Europe and Asia Our products excel where high strength light weight and durability are required TPI has over five decades of innovation and expertise in manufacturing largescale composite structures of highest quality at a competitive total delivered cost to our customersOur advanced composites manufacturing technology allows us to build near aerospace grade parts at industrial prices TPIs advanced composite technology and manufacturing expertise have also been applied to create firstoftheirkind composite solutions in the transportation markets"
sentiment_scores(sentence)

print("\n52 :")
sentence = "Mister Car Wash is the largest and fastest growing car wash chain in the United States with over 300 locations 32 Oil Change Express centers and more than 7500 employees The growth of Mister Car Wash has been fueled by innovative technology valued customers and investors and most importantly motivated employees The Mister Car Wash brand is anchored in quality speed and our commitment to creating a memorable customer experience We deliver a clean dry shiny car but what really sets us apart is our people The mister car wash brand is about inspiring potential and giving both our employees and customers the motivation to be their best The people who choose to work at Mister Car Wash are not just looking for a job; theyre interested in building a career with a growing company We dont just believe in shiny cars but giving our employees the opportunity to shine as wellJoin our team and shine with us"
sentiment_scores(sentence)

print("\n53 :")
sentence = "First Solar Nasdaq FSLR is a leading global provider of comprehensive photovoltaic PV solar systems which use its advanced thinfilm modules The companys integrated power plant solutions deliver an economically attractive alternative to fossilfuel electricity generation today From raw material sourcing through endoflife module collection and recycling First Solars renewable energy systems protect and enhance the environment "
sentiment_scores(sentence)

print("\n54 :")
sentence = "Kona Grills Mission is to make every experience exceptional We pride ourselves on offering our guests; * Fresh ingredients and entrees made from scratch with passion by our executive chefs and their teams * Fresh fish flown in daily to deliver outstanding sushi quality * Genuine hospitality with personality * Specialty cocktails served in an energetic bar and lounge * An upscalecasual environment that provides the perfect backdrop for a date a party and everything in between We look forward to the growth of our business and your career Contact us today "
sentiment_scores(sentence)

print("\n55 :")
sentence = "Forever Living Products and its affiliates are the largest growers manufacturers and distributors of aloe vera products in the world The key to Forevers success is commitment to quality and purity With offices in over 160 countries and Forever Business Owners worldwide our goal is to provide a healthier life for everyone Its in our nature to share good things Tell a friend about the Forever Products and spread the healthy benefits Thats how our social business works One story at a time One person to the next To learn more about our products and the Forever Opportunity go to ForeverLivingcom Be sure to follow us on Facebook facebookcom/ForeverGlobalHQ Twitter twittercom/ForeverGlobalHQ and LinkedIn linkedincom/company/foreverlivingcorporate"
sentiment_scores(sentence)

print("\n56 :")
sentence = "Cable One Inc NYSE CABO is a leading broadband communications provider serving more than 900000 residential and business customers in 21 states through its Sparklight™ and Clearwave brands Sparklight provides consumers with a wide array of connectivity and entertainment services including highspeed internet and advanced WiFi solutions cable television and phone service Sparklight Business and Clearwave provide scalable and costeffective products for businesses ranging in size from small to midmarket in addition to enterprise wholesale and carrier customers"
sentiment_scores(sentence)

print("\n57 :")
sentence = "Mobile Mini Inc is the worlds leading provider of portable storage solutions through its total lease fleet of approximately 213500 portable storage containers and office units with 136 locations in the US United Kingdom and Canada Were honored to be named one of Arizonas Most Admired Companies for 2018 by AZ Business Magazine This prestigious award recognizes excellence in workplace culture leadership social responsibility and customer opinion Through its whollyowned subsidiary Evergreen Tank Solutions Mobile Mini is also the third largest provider of specialty containment solutions in the US with a lease fleet of approximately 10600 units and 24 locations Mobile Mini is included on the Russell 2000® and 3000® Indexes and the S&P Small Cap Index For more information visit wwwmobileminicom"
sentiment_scores(sentence)

print("\n58 :")
sentence = "A multipletime recipient of Air Transport Worlds Regional Airline of the Year Award Phoenixbased Mesa Airlines operates as American Eagle from hubs in Phoenix and Dallas/Fort Worth and as United Express from Washington Dulles and Houston Founded on a mesa in New Mexico in 1982 by Larry and Janie Risley Mesa currently operates 145 large regional jets and more than 700 daily system departures to 100 cities 44 states the District of Columbia Canada and Mexico The company has approximately 3500 employees and plans to hire an additional 1000 in the next two years Visit our website at wwwmesaaircom"
sentiment_scores(sentence)

print("\n59 :")
sentence = "Supplying IT Solutions Tailored to Our Clients? Specific Business Needs Helping you to make competent and confident decisions regarding your computer network by taking you stepbystep through a needs analysis followed by a detailed customized proposal Our mission is to uphold the highest standards in Personalized and Professional IT Services As our IT Solutions Provider we find an Information Technology solution that works deliver it quickly and costeffectively ensuring you derive real and measurable business value from it Contact us to learn about our IT services and request your free needs analysis"
sentiment_scores(sentence)

print("\n60 :")
sentence = "Qwick is changing the way people work  We connect food & beverage professional who want ondemand work with businesses who need them  Qwick offers our team members flexibility control over their schedules and the ability to choose who they want to work for  Businesses benefit by being able to fill shifts in realtime with a qualified and vetted workforce at a fraction of the cost of traditional staffing agencies"
sentiment_scores(sentence)

print("\n61 :")
sentence = "Knight Transportation Inc is a truckload carrier offering dry van refrigerated intermodal and brokerage services to customers through a network of service centers and branches located throughout the United States serving North America As “Your Hometown National Carrier” Knight strives to offer customers and drivers personal service and attention through each service center while offering integrated freight transportation nationwide and beyond through the scale of one of North Americas largest trucking companies The principal types of freight we transport include consumer staples retail paper products packaging/plastics manufacturing and import/export commodities"
sentiment_scores(sentence)

print("\n62 :")
sentence = "Were more than a multifamily development and management company Here at Alliance were a family of entrepreneurs and lifestyle ambassadors with a shared mantra of respect always We value partnership and operate under the belief that the collective whole is better than the sum of its parts Its because of this strong foundation that were able to build opportunities create homes and grow leaders of unparalleled quality Headquartered in Phoenix with 35 regional offices nationwide and with a portfolio of 350 clients and counting were in constant search of passionate people to join our team Limitless training programs A clear roadmap for your professional career Countless perks And endless opportunities for growth Thats what you get when you join Alliance Lets build something great together"
sentiment_scores(sentence)

print("\n63 :")
sentence = "We are Paradox the AI company that believes recruiting is a people game  Our flagship product is Olivia the AI recruiting assistant obsessed with improving and reinventing the candidate experience Olivia helps companies capture and screen candidates improve conversions and answer all candidate questions  She delivers onetoone candidate experience at scale and even handles interview scheduling At Paradox we see a future where humans do the things theyre best at and AI technology handles the mundane and simple tasks for us  Technology is a tool for us and should be used as such We never want to remove humans from the recruiting process  We just want to make it better Our dream is a better candidate and recruiter experience for everyone"
sentiment_scores(sentence)

print("\n64 :")
sentence = "Hivewyre is a forwardthinking programmatic advertising company Their private data cooperative allows advertisers to share firstparty audience data and build targeted campaigns for prospecting new customers based on their competitors unconverted shoppers Hivewyre advertisers have access to fresh exclusive data that will never be sold to a third party With over 1000 retailers and over 100 million monthly inmarket shoppers in our coop our distinct advertising strategy allows you to combat rising search engine PPC & PLA costs and the everexpanding online marketplace growth In addition to access to the exclusive data coop members benefit from fullservice account management and detailed reporting of the return from advertising spend"
sentiment_scores(sentence)

print("\n65 :")
sentence = "iMemories is a pioneer in social entertainment reinventing how family memories are experienced in todays cloudconnected multiscreen world iMemories is a cloudbased storage and streaming service that puts a lifetime of memories at your fingertips From vintage home movies and slides to modern digital videos and photos your familys entire collection of personal videos and photos is always with you organized and ready to watch in brilliant clarity on nearly any screen iMemories is available on iPhone iPad Android phone and tablet PC and Mac and select Smart TVs"
sentiment_scores(sentence)


print("\n66 :")
sentence = "Todays consumers are savvy They want highly personalized offers and promotions delivered at the right time and place But most local business owners lack the time and expertise needed to do this effectively quantifiably and affordably without being gouged on profit So they spend thousands of dollars on generic marketing methods and tools that produce lackluster  if any  results Hownd™ revolutionizes how entertainment health & beauty food & beverage and other local businesses generate more foot traffic Our fully automated consumer behaviordriven marketing platform creates effortless foot traffic more repeat visits instant revenue and higher profit  and its all quantifiable We do it by displaying targeted Buy Now and other promotional offers on your website via email and social media WiFi data capture and our own MyHownd™ mobile app  at times when consumers are most likely to purchase It only takes 5 minutes to set up and is zerorisk because you only pay when we bring you customers"
sentiment_scores(sentence)

print("\n67 :")
sentence = "EpiFinder mission is to reduce misdiagnosis and disrupt health care Our proprietary software in research and development aspires to provide a screening tool for neurologists who want to know more about and effectively diagnose epilepsy syndromes at the pointofcare EpiFinder has a universitybased internship credit program for intern team members "
sentiment_scores(sentence)

print("\n68 :")
sentence = "Promotion Vault is the leading provider of electronic gift cards for the Fitness IndustryPromotion Vault communicates on your behalf with customers who have selected a gift card option and provides 24/7 support if anyone needs assistance in taking advantage of their reward"
sentiment_scores(sentence)

print("\n69 :")
sentence = "Yazamo is a SaaS marketing company that generates leads and sales for small to midsize businesses Our mission is to give businesses a platform to develop optimize and scale their online marketing efforts"
sentiment_scores(sentence)

print("\n70 :")
sentence = "The Akwai mobile app is a social network thats based around goals and accountability It has a marketplace with thousands of Action Plans that guide you on accomplishing specific objectives We have Action Plans for any goal you can think of  parenting fitness finances… weve even converted books into Action Plans As you use Akwai the app collects data and measures attributes that are essential to success Each week it pairs you with your groupmembers for guided checkins"
sentiment_scores(sentence)


print("\n71 :")
sentence = "At scale Ampsy uses its proprietary social geofencing technology and machine learning to help clients identify segment & activate their most socially influential customers at events or business locations leading to increased brand loyalty engagement & revenue Hundreds of clients have used the Ampsy platform including the Houston Super Bowl Host Committee Jimmy Kimmel Live Red Bull Washington Wizards & Capitals Phoenix Suns AT&T Samsung Linkin Park and many more Ampsy was previously a Top 8 finalist at the SXSW Accelerator Competition and subsequently named one of the 20 Most Interesting Technologies at SXSW by IPG Media Lab Ampsy is backed by Exponential Partners Scout Ventures Arab Angel Fund Hivers & Strivers Rick Dalzell former Amazon CIO and Raul Fernandez Vice Chairman Monumental Sports & Entertainment"
sentiment_scores(sentence)

print("\n72 :")
sentence = "Aquimo is a US based mobile games development company having a wholly owned subsidiary Aquimo Sports Private Limited in India It was founded by Kellogg School of Management Professor Mark Jeffery technology guru Manoj Rana and creative genius Rob KomourousKing with the help of Mike Malaska 2011 PGA Golf Instructor of the Year Our team which consists of highly talented passionate and successful individuals have developed a unique and disruptive technology that allows users to use their smart phones to swing like sport equipment eg golf club tennis racket baseball bat etc Coupled with mobile gaming apps the plays can be viewed in real time on smart phones or any other Internet enabled viewing device TV computer tablet etc and can be used for practice or to take lessons or to compete with other users in real time Users can play competitive games with multiple friends located anywhere in the world with each player being able to see others shots and converse in real time or take lessons from pros based on the swing analysis We launched our first golf game Pin High followed by other sports like tennis baseball bowling etc Our vision is to transform mobile sports gaming Company has filed for multiple US and International patents to protect its invention"
sentiment_scores(sentence)


print("\n73 :")
sentence = "Aspireship is a career development and hiring platform for the SaaS industry We train candidates for highpaying careers in sales account management and other gotomarket specialties then connect top performing candidates with growing companies nationwide"
sentiment_scores(sentence)


print("\n74 :")
sentence = "Avontage is more than just a marketplace its a vibrant community powered by local business owners freelancers entrepreneurs creatives digital marketers influencers and bloggers who collaborate build longlasting connections and help one another to grow their businesses"
sentiment_scores(sentence)

print("\n75 :")
sentence = "DeliverSense is a cloudbased software platform comprised of tools that are designed to automate redundant and complex tax and accounting processes Users of the platform can pick and choose which tools they want to use and will only pay for how much they use the toolsOur Mission To enable tax and accounting teams across industries to reduce their reliance on errorprone spreadsheets by providing cloudbased automation tools at a price that makes sense for any company"
sentiment_scores(sentence)

print("\n76 :")
sentence = "Black Pearl Mail was launched in 2014 Our goal was to make email a more effective business tool We felt that emails should look better be easier to manage and more transparent As we evolved we realized that attractive dynamic emails created increased web and social media traffic from recipients clicking links and images in the email thus email as an owned media channel was born  or as we like to call it Owned Email™ We are extremely proud of our company We have unique technology which we believe is critical for both effective branding/management of email and using email as an owned media channel Drop us an email if you want to talk business distribution employment or the wonderful world of email in general"
sentiment_scores(sentence)


print("\n77 :")
sentence = "Quickly and easily build and categorize logbooks manuals photos Add multiple aircraft to your digital hangar esign documents privately share records and more Our features were built to be powerful alone but even stronger together "
sentiment_scores(sentence)

print("\n78 :")
sentence = "ElderSense is an assisted directory and digital marketing tool focused on smaller adult group homes licensed for 20 beds or less that make up 68% of industry beds While the top 10 assisted living companies represent about a third of the market our clients are the largest market segment but have the least depth of resources personnel and tools to help them compete against the competition We provide them with a full service platform to reach consumers list of caregivers to fill staff openings and digital marketing and SEO tools allowing them to stand shoulder to shoulder with their larger resourced competition"
sentiment_scores(sentence)


print("\n79 :")
sentence = "Emagine Solutions Technology improves patient care by shrinking an ultrasound to the size of a phone so womens health workers of any skill level can quickly scan a patient anytime and anywhere Our VistaScan handheld ultrasound is a clinicians first line of defense to quickly understand an OBGYN or emergency patients condition Clinicians of all skill levels can scan and diagnose patients at the bedside or in transit in moments saving time and ultimately lives at 10% the cost a cart ultrasound"
sentiment_scores(sentence)


print("\n80 :")
sentence = "EndoVantage provides clinicians with novel computational tools for the personalized planning of endovascular treatments Using EndoVantages tools clinicians can evaluate different endovascular devices design treatment strategies and compare treatment costs for each unique patient case Through these unprecedented treatment planning capabilities clinicians can design the optimal treatment strategy on a casebycase basis"
sentiment_scores(sentence)

print("\n81 :")
sentence = "eTracks SaaS solution provides IoTenabled predictive insights for heavy equipment to eliminate downtime reduce repair costs improve safety and provide strategic asset tracking BI eTrack Tech Inc launched at the Founder Institute"
sentiment_scores(sentence)


print("\n82 :")
sentence = "eUnify is a software shop dedicated to the Community Association industry We saw the need for software done right in this industry way back in 2001 and have built a universe of products apps and functions since then Today eUnify offers an allinone solution including a full accounting platform with banking integration; management tools and mobile app for architectural review covenant enforcement and work orders; secure portals for residents boards and vendors; and a full resale document order and production product Visit our website wwweUnifynet to learn more about our products and bookmark our blog the ‘HOA Tech Insider infoeunifynet/blog which is a great educational resource You can always call us tollfree at 18664eUnify 18664386439 or send an email to sales@eUnifynet"
sentiment_scores(sentence)


print("\n83 :")
sentence = "Fasetto creates technology solutions for an increasingly connected world We believe in seamless connectivity between people their content and the devices they use regardless of platform device or network"
sentiment_scores(sentence)


print("\n84 :")
sentence = "Founded in 2013 Find Your Influence is the leading SaaSbased influencer marketing solution built by marketers for marketers Relied on by many of the top brands across the United States FYI leverages proprietary technology for discovering influencers managing campaigns and tracking metrics In an everevolving digital marketplace FYI manages relationships with brands and pairs them with the right influencers to deliver guaranteed results FYI is headquartered in Scottsdale Arizona and is led by cofounders Jamie Reardon and Cristine Vieira"
sentiment_scores(sentence)


print("\n85 :")
sentence = "GroveSite offers “PLM Sourcing” a suite of easy PLM software tools to help retailers and farflung suppliers collaborate to plan next seasons product assortments GroveSites configurable online product database provides a single source of product development information that is organized uptodate and accessible to your teams 24/7 via your browser It is a great way to share specifications securely collect quotes request samples and make final product and vendor selections Equally important GroveSites PLM Sourcing tools are very affordable making us an alternative to multimillion dollar PLM solutions Our PLM Sourcing software tools are hosted on GroveSites ‘private cloud making deployment quick and easy GroveSite is secure affordable and backed up by an unparalleled customer service team Sign up for a brochure and request a free demo or trial using the link below"
sentiment_scores(sentence)


print("\n86 :")
sentence = "Kozees mission is to provide homeowners with ease of mind and simplicity with an allinone mobile application and platform for their homes  The ability to access a Kozee Rating using unique algorithms machine learning and artificial intelligence give homeowners the ability to understand their home and what services are necessary  With Kozee homeowners will now be able to store and transfer home service records request home services and repairs from trusted Kozee Certified contractors and connect to popular smart home applications Kozee was founded by two millennials that purchased their own homes and saw the deep flaws and gaps in the home service industry  After being disrespected by shady contractors the founders saw a huge opportunity to improve and disrupt the Home Services industry to make the world a happier place for homeowners  The founders of Kozee have instilled the values of Transparency Trust and Teamwork into the Kozee Kore Values and look to grow the company with strong leaders and passionate people with a customer first mindset Kozee is headquartered in Phoenix Arizona and will be expanding all throughout the US Know your Home  Control Your Home  Get Kozee"
sentiment_scores(sentence)


print("\n87 :")
sentence = "Life365 is a leading telehealth and remote care management solution provider The Life365 digital health platform includes solutions for patients with various conditions where remote patient monitoring and active selfmanagement will result in better health outcomes Our solutions gather biometric measurements and other data from patients located in their homes Better patient insights mean more informed care and faster interventions that help avoid costly admissions and readmissions Whether our customers need to manage patient populations or reach success with new payment models  our connected technology can help them meet their goals Our product offerings include Solutions for patients  which include cellular enabled tablets smartphones with preinstalled applications and connected medical devices to gather vitals Our connectivity solutions promote patient engagement while delivering valuable health information to clinicians Clinical monitoring portal for clinicians is an online web based portal that allows clinicians to review patient health data produce reports and assign patients to staff for monitoring Clinicians can identify patients that need the most attention and provide timely interventions if a patient shows signs of clinical deterioration Life365 also offers services to ensure successful and sustainable telehealth programs  including logistics software integrations and healthcare staffing available for in person and / or virtual monitoring patients Life365s CoFounders have extensive experience developing healthcare technology solutions Their previous startup MedApps was acquired by Alere now Abbott Laboratories for 30x revenue in 2012"
sentiment_scores(sentence)




51 :
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.806, 'pos': 0.194, 'compound': 0.9601}
sentence was rated as  0.0 % Negative
sentence was rated as  80.60000000000001 % Neutral
sentence was rated as  19.400000000000002 % Positive
Sentence Overall Rated As Positive

52 :
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.752, 'pos': 0.248, 'compound': 0.9907}
sentence was rated as  0.0 % Negative
sentence was rated as  75.2 % Neutral
sentence was rated as  24.8 % Positive
Sentence Overall Rated As Positive

53 :
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.793, 'pos': 0.207, 'compound': 0.8834}
sentence was rated as  0.0 % Negative
sentence was rated as  79.3 % Neutral
sentence was rated as  20.7 % Positive
Sentence Overall Rated As Positive

54 :
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.76, 'pos': 0.24, 'compound': 0.9755}
sentence was rated as  0.0 % Negative
sentence was rated as  76.0 % Neutral
sentence was rated as  24.0 % Positive
Se